In [ ]:
import h5py
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import copy
from typing import List, Dict
import pandas as pd
plt.style.use('ggplot')
# plt.rcParams["figure.figsize"] = (12,10)
plt.rcParams["figure.figsize"] = (10,8)

# Code to read Horizons.h5

In [ ]:
def make_Bh_pandas(h5_dir):
    # Empty dataframe
    df = pd.DataFrame()
    
    # List of all the vars in the h5 file
    var_list = []
    h5_dir.visit(var_list.append)
    
    
    for var in var_list:
        # This means there is no time column
        # print(f"{var} : {h5_dir[var].shape}")
        if df.shape == (0,0):
            # data[:,0] is time and then we have the data
            data = h5_dir[var]
            
            # vars[:-4] to remove the .dat at the end
            col_names = make_col_names(var[:-4],data.shape[1]-1)
            col_names.append('t')
            # Reverse the list so that we get ["t","var_name"]
            col_names.reverse()            
            append_to_df(data[:],col_names,df)
            
        else:
            data = h5_dir[var]
            col_names = make_col_names(var[:-4],data.shape[1]-1)         
            append_to_df(data[:,1:],col_names,df)
            
    return df

def append_to_df(data,col_names,df):
    for i,col_name in enumerate(col_names):
        df[col_name] = data[:,i]
        
def make_col_names(val_name:str,val_size:int):
    col_names = []
    if val_size == 1:
        col_names.append(val_name)
    else:
        for i in range(val_size):
            col_names.append(val_name+f"_{i}")
    return col_names


def horizon_to_pandas(horizon_path:Path):
    assert(horizon_path.exists())
    df_dict = {}
    with h5py.File(horizon_path,'r') as hf:
        # Not all horizon files may have AhC
        for key in hf.keys():
            df_dict[key[:-4]] = make_Bh_pandas(hf[key])

    return df_dict

def read_horizon_across_Levs(path_list:List[Path]):
    df_listAB = []
    df_listC = []
    final_dict = {}
    for path in path_list:
        df_lev = horizon_to_pandas(path)
        # Either [AhA,AhB] or [AhA,AhB,AhC]
        if len(df_lev.keys()) > 1:
            df_listAB.append(df_lev)
        # Either [AhC] or [AhA,AhB,AhC]
        if (len(df_lev.keys()) == 1) or (len(df_lev.keys()) ==3):
            df_listC.append(df_lev)
    if len(df_listAB)==1:
        # There was only one lev
        final_dict = df_listAB[0]
    else:
        final_dict["AhA"] = pd.concat([df["AhA"] for df in df_listAB])
        final_dict["AhB"] = pd.concat([df["AhB"] for df in df_listAB])
        if len(df_listC) > 0:
            final_dict["AhC"] = pd.concat([df["AhC"] for df in df_listC])       
    
    return final_dict

def moving_average(array,avg_len):
    return np.convolve(array,np.ones(avg_len))/avg_len
    
def moving_average_valid(array,avg_len):
    return np.convolve(array,np.ones(avg_len),'valid')/avg_len

def plot_graph_for_runs(runs_data_dict, x_axis, y_axis, minT, maxT, save_path=None, moving_avg_len=0, plot_fun = lambda x,y,label : plt.plot(x,y,label=label)):

  minT_indx_list={}
  maxT_indx_list={}
  
  for run_name in runs_data_dict.keys():
    minT_indx_list[run_name] = len(runs_data_dict[run_name][x_axis][runs_data_dict[run_name][x_axis] < minT])
    maxT_indx_list[run_name] = len(runs_data_dict[run_name][x_axis][runs_data_dict[run_name][x_axis] < maxT])

  if moving_avg_len == 0:

    for run_name in runs_data_dict.keys():
      x_data = runs_data_dict[run_name][x_axis][minT_indx_list[run_name]:maxT_indx_list[run_name]]
      y_data = runs_data_dict[run_name][y_axis][minT_indx_list[run_name]:maxT_indx_list[run_name]]
      plot_fun(x_data, y_data,run_name)

    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    title = "\"" +  y_axis+"\" vs \""+x_axis+"\""
    plt.title(title)
    plt.legend()

  else:
    for run_name in runs_data_dict.keys():
      x_data = runs_data_dict[run_name][x_axis][minT_indx_list[run_name] + moving_avg_len-1:maxT_indx_list[run_name]]
      y_data = moving_average_valid(runs_data_dict[run_name][y_axis][minT_indx_list[run_name]:maxT_indx_list[run_name]], moving_avg_len)
      plot_fun(x_data, y_data,run_name)

    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    title = "\"" + y_axis+ "\" vs \"" + x_axis + "\"  " + f"avg_window_len={moving_avg_len}"
    plt.title(title)
    plt.legend()

  
  if save_path is not None:
    fig_x_label = x_axis.replace("/","_").replace(".","_")
    fig_y_label = y_axis.replace("/","_").replace(".","_")
    save_file_name = f"{fig_y_label}_vs_{fig_x_label}_minT={minT}_maxT={maxT}_moving_avg_len={moving_avg_len}"
    for run_name in runs_data_dict.keys():
      save_file_name = save_file_name + "__" + run_name

    plt.savefig(save_path+save_file_name)

def load_data_from_levs(base_path:Path, runs_path:Dict[str,Path]):
  data_dict = {}
  for run_name in runs_path.keys():
    path_list = list(base_path.glob(runs_path[run_name]))
    print(path_list)
    data_dict[run_name] = read_horizon_across_Levs(path_list)
  return data_dict

def flatten_dict(horizon_data_dict:Dict[str,pd.DataFrame]) -> Dict[str,pd.DataFrame] :
  flattened_data = {}
  for run_name in horizon_data_dict.keys():
      for horizons in horizon_data_dict[run_name]:
          flattened_data[run_name+"_"+horizons] = horizon_data_dict[run_name][horizons]
          # print(run_name+"_"+horizons)
  return flattened_data

In [ ]:
runs_to_plot = {}
base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs")
runs_to_plot["76_ngd_master_mr1_50_3000"] =  "76_ngd_master_mr1_50_3000/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["76_ngd_master_mr1_200_3000"] =  "76_ngd_master_mr1_200_3000/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["77_gd_Kerr_q1"] =  "77_gd_Kerr_q1/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["77_gd_Kerr_q3"] =  "77_gd_Kerr_q3/Ev/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["77_gd_Kerr_q1_Kerr"] =  "77_gd_Kerr_q1/Ev_Kerr/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["77_gd_Kerr_q3_Kerr"] =  "77_gd_Kerr_q3/Ev_Kerr/Lev1_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["78_ngd_master_mr1"] =  "78_ngd_master_mr1/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["79_ngd_master_mr1_1000_3000"] =  "79_ngd_master_mr1_1000_3000/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["79_ngd_master_mr1_200_3000"] =  "79_ngd_master_mr1_200_3000/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_100"] =  "80_ngd_master_mr1_100/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_50"] =  "80_ngd_master_mr1_50/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_10"] =  "80_ngd_master_mr1_10/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_5"] =  "80_ngd_master_mr1_5/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["80_ngd_master_mr1_300"] =  "80_ngd_master_mr1_300/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["81_gd_Kerr_q3_0_9_0__0_0_0"] =  "81_gd_Kerr_q3_0_9_0__0_0_0/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["81_gd_DH_q3_0_9_0__0_0_0"] =  "81_gd_DH_q3_0_9_0__0_0_0/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["82_ngd_master_mr1_50_3000_DH_to_DH"] =  "82_ngd_master_mr1_50_3000_DH_to_DH/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_ngd_master_mr1_200_3000_no_eps"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_ngd_master_mr1_200_3000_no_eps_no_lsr"] =  "83_ngd_master_mr1_200_3000_no_eps_no_lsr/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["84_gd_KerrI_3000_200"] =  "84_gd_KerrI_3000_200/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["84_gd_DH_3000_200"] =  "84_gd_DH_3000_200/Ev/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_wrong_evolution"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_wrong_evolution/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["83_no_eps_Ev_pow2"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_pow2/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
runs_to_plot["83_no_eps_Ev_pow6"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_pow6/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_tanh15"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_tanh15/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_tanh7"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_tanh7/Lev3_A?/Run/ApparentHorizons/Horizons.h5"
# runs_to_plot["83_no_eps_Ev_tanh7_lsr_correct_evolution"] =  "83_ngd_master_mr1_200_3000_no_eps/Ev_tanh7_lsr_correct_evolution/Lev3_A?/Run/ApparentHorizons/Horizons.h5"

data_dict = load_data_from_levs(base_path, runs_to_plot)
data_dict = flatten_dict(data_dict)
data_dict[list(data_dict.keys())[0]].columns

In [ ]:
moving_avg_len = 0
save_path = None

x_axis = 't'
# y_axis = 'ArealMass'
# y_axis = 'ChristodoulouMass'
# y_axis = 'CoordCenterInertial_0'
# y_axis = 'CoordCenterInertial_1'
# y_axis = 'CoordCenterInertial_2'
# y_axis = 'DimensionfulInertialSpin_0'
# y_axis = 'DimensionfulInertialSpin_1'
# y_axis = 'DimensionfulInertialSpin_2'
y_axis = 'DimensionfulInertialSpinMag'
# y_axis = 'SpinFromShape_0'
# y_axis = 'SpinFromShape_1'
# y_axis = 'SpinFromShape_2'
# y_axis = 'SpinFromShape_3'
# y_axis = 'chiInertial_0'
# y_axis = 'chiInertial_1'
# y_axis = 'chiInertial_2'
# y_axis = 'chiMagInertial'



# moving_avg_len=25
minT = 2500
maxT = 5000

plot_fun = lambda x,y,label : plt.plot(x,y,label=label)
# plot_fun = lambda x,y,label : plt.semilogy(x,y,label=label)
# plot_fun = lambda x,y,label : plt.loglog(x,y,label=label)
# plot_fun = lambda x,y,label : plt.scatter(x,y,label=label)
# save_path = "/panfs/ds09/sxs/himanshu/scripts/report/not_tracked/temp2/"

filtered_dict = {}
allowed_horizons = ["AhA"]
for horizons in allowed_horizons:
  for runs_keys in data_dict.keys():
    if horizons in runs_keys:
      filtered_dict[runs_keys] = data_dict[runs_keys]

with plt.style.context('default'):
  plt.rcParams["figure.figsize"] = (12,10)
  plt.rcParams["figure.autolayout"] = True
  plot_graph_for_runs(filtered_dict, x_axis, y_axis, minT, maxT, save_path=save_path, moving_avg_len=moving_avg_len, plot_fun=plot_fun)

In [ ]:
base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/83_ngd_master_mr1_200_3000_no_eps/Ev_pow2")
base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/83_ngd_master_mr1_200_3000_no_eps/Ev_pow6")
# base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/76_ngd_master_mr1_50_3000/Ev")
# base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/76_ngd_master_mr1_200_3000/Ev")
# base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/79_ngd_master_mr1_200_3000/Ev")
# base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/79_ngd_master_mr1_1000_3000/Ev")
file_pattern = "Lev3_A?/Run/ApparentHorizons/Horizons.h5"
path_list = list(base_path.glob(file_pattern))
path_list

In [ ]:
df = read_horizon_across_Levs(path_list)
print(df.keys())
df["AhA"].keys()

In [ ]:
x = 't'
y = "SpinFromShape_2"
plt.semilogy(df['AhA'][x],df['AhA'][y],label="AhA")
plt.semilogy(df['AhB'][x],df['AhB'][y],label="AhB")
# plt.plot(df['AhC'][x],df['AhC'][y],label="AhC")
plt.xlabel(x)
plt.ylabel(y)
plt.legend()

In [ ]:
plt.plot(df['AhC'][x],df['AhC'][y],label="AhC")
plt.xlabel(x)
plt.ylabel(y)
plt.legend()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df["AhA"].describe())